<a href="https://colab.research.google.com/github/gokulanv/Data_Shapley_NLP/blob/master/AFLite_mimic_covtype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer, fetch_covtype
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### One time config

In [ ]:
data, labels = fetch_covtype(return_X_y = True)
data = data[:30000]
labels = labels[:30000]
train_size = 15000
predictability_score = [0.0] * data.shape[0]
m = 10

classifiers = [
               LogisticRegression(n_jobs=4, C=1.0, max_iter=5000, random_state=666),
               OneVsRestClassifier(SVC(random_state=666), n_jobs=4)
               ]

pipe = Pipeline([
                 ('normalizer', StandardScaler()),
                 ('clf', None)
])

#Check if all data points are unique
unique_rows = np.unique(data, axis=0)
assert unique_rows.shape == data.shape

# TODO: Remove duplicate points, if any

data2idx = dict({tuple(v):k for k,v in enumerate(data)})
idx2count_sampled_points = dict({k: 0 for k in range(data.shape[0] + 1)})
idx2count_classified_correctly = dict({k: 0 for k in range(data.shape[0] + 1)})

# check if indexed properly
test_indices = np.random.randint(0,data.shape[0], 10)
for idx in test_indices:
    assert data2idx[tuple(data[idx])] == idx

assert sum(idx2count_sampled_points[k] for k in idx2count_sampled_points.keys()) == 0
assert sum(idx2count_classified_correctly[k] for k in idx2count_sampled_points.keys()) == 0
assert len(idx2count_classified_correctly) == len(idx2count_sampled_points)


In [ ]:
train_size = 15000
np.unique(labels[:10000])

array([1, 2, 3, 4, 5, 6, 7], dtype=int32)

### Utils

In [ ]:
def get_random_train_test_split(train_size: int) -> tuple:
    return train_test_split(data, labels, train_size = train_size)

def update_test_datapoints_count(Xtest: np.ndarray, idx2count: dict, indices: list = None) -> None:
    if indices:
        # update only given indices
        Xtest = Xtest[indices]
    [idx2count.__setitem__(data2idx[tuple(dp)], idx2count[data2idx[tuple(dp)]] + 1)  for dp in Xtest]

# update_test_datapoints_count(Xtest, idx2count_sampled_points)

### Models pipeline

In [ ]:
# AFLite random sampling
for i in range(m):
    Xtrain, Xtest, Ytrain, Ytest = get_random_train_test_split(train_size)

    for clf in classifiers:
        pipe.set_params(clf = clf)
        # scores = cross_validate(pipe, data, labels)
        grid_search = GridSearchCV(pipe, {}, n_jobs=4)
        grid_search.fit(Xtrain, Ytrain)

        correct_pred_indices = [i for i in range(len(Xtest)) if grid_search.predict(np.expand_dims(Xtest[i], axis=0)) == Ytest[i]]
        update_test_datapoints_count(Xtest, idx2count_classified_correctly, indices = correct_pred_indices)

    for i in range(len(predictability_score)):
        predictability_score[i] += idx2count_classified_correctly[i]/(idx2count_sampled_points[i] + 1e-9)
        

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
print('#data_points < 100 test score threshold = ', str(np.sum(np.asarray(predictability_score) < 100)) )

#data_points < 100 test score threshold =  4432


In [ ]:
np.sum(np.asarray(predictability_score) == 0)

4432

### Logistic Regression

In [ ]:
a = np.array([[10, 7, 4], [3, 2, 1]])
q = [25,50,75, 100]

[2.25, 3.5, 6.25, 10.0]